In [1]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 30.8 MB/s 
     |████████████████████████████████| 9.3 MB 44.1 MB/s 
     |████████████████████████████████| 1.6 MB 42.2 MB/s 
     |████████████████████████████████| 108 kB 61.2 MB/s 


In [2]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [3]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [4]:
df.drop(['packer_type'],axis=1,inplace=True)

In [5]:
from sklearn.utils import shuffle
df = shuffle(df)

In [6]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [7]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [8]:
df.drop(['index'],axis=1,inplace=True)

In [9]:
df_5percent=df[0:1823] 


In [10]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,240,6,1,0,1,...,1,2,4,0,6.496799,0.000000,92160,7.659083,1,1
1,144,8,4,65535,40,128,5,1,1,1,...,1,1,4,0,5.960037,5.062149,61440,6.123409,0,1
2,144,3,4,65535,184,224,4,1,0,1,...,1,1,3,0,6.244721,3.252343,46080,6.106604,1,1
3,144,3,4,65535,184,224,4,1,0,1,...,1,1,3,1,6.607069,0.666147,40960,4.279894,0,0
4,144,3,4,65535,184,256,5,1,0,1,...,1,1,4,0,6.493486,5.761705,1160528,6.420332,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1818,144,3,4,65535,184,128,3,1,0,1,...,1,1,2,1,5.427656,0.000000,7168,4.697734,0,0
1819,144,3,4,65535,184,224,3,1,0,1,...,1,0,3,0,7.258754,7.947003,254464,7.518498,0,1
1820,144,3,4,65535,184,192,3,1,1,1,...,1,2,1,1,0.000000,0.000000,47616,7.864474,1,1
1821,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,0,6.506643,0.462952,45056,6.389507,1,0


In [11]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (869, 69)
class 1: (954, 69)


In [12]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [13]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0      144.0   3.0        4.0     65535.0  184.0     240.0               6.0   
1      144.0   8.0        4.0     65535.0   40.0     128.0               5.0   
2      144.0   3.0        4.0     65535.0  184.0     224.0               4.0   
3      144.0   3.0        4.0     65535.0  184.0     224.0               4.0   
4      144.0   3.0        4.0     65535.0  184.0     256.0               5.0   
...      ...   ...        ...         ...    ...       ...               ...   
1818   144.0   3.0        4.0     65535.0  184.0     128.0               3.0   
1819   144.0   3.0        4.0     65535.0  184.0     224.0               3.0   
1820   144.0   3.0        4.0     65535.0  184.0     192.0               3.0   
1821   144.0   3.0        4.0     65535.0  184.0     224.0               5.0   
1822    80.0   2.0        4.0     65535.0  184.0     256.0               9.0   

      C

In [14]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,1823.000000,1823.000000,1823.000000,1823.000000,1823.000000,1823.000000,1823.000000,1823.000000,1823.000000,1823.0,...,1823.000000,1823.000000,1823.000000,1823.000000,1823.000000,1823.000000,1.823000e+03,1823.000000,1823.000000,1823.000000
mean,148.605047,13.907844,11.014262,65114.326385,228.558420,222.051563,4.686231,0.990126,0.357104,1.0,...,0.998903,1.413055,3.273176,0.163467,4.948438,2.493674,6.900277e+05,6.329889,0.537575,0.523313
std,412.653360,469.524356,300.891429,5187.847032,1584.788361,48.048224,1.967704,0.098903,0.479277,0.0,...,0.033113,1.689376,1.127434,0.369893,2.523527,2.698584,4.264538e+06,1.155392,0.498723,0.499593
min,0.000000,0.000000,0.000000,0.000000,0.000000,16.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.942085,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,3.890014,0.000000,6.173800e+04,5.667680,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.147736,1.454411,1.208320e+05,6.366100,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.507063,4.637715,2.772440e+05,7.304952,1.000000,1.000000
max,17739.000000,20050.000000,12851.000000,65535.000000,65534.000000,592.000000,34.000000,1.000000,1.000000,1.0,...,1.000000,31.000000,7.000000,1.000000,7.998838,7.999568,8.867701e+07,7.999983,1.000000,1.000000


In [15]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [16]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [17]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [18]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
1.0      52.5
0.0      47.5
dtype: float64

In [19]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
0.0      55.0
1.0      45.0
dtype: float64

In [20]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [21]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [22]:
circuit(weights_init, data[0])

tensor(0.02294742, requires_grad=True)

In [23]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias


In [24]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss


In [25]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [26]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [27]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.295146, requires_grad=False), tensor(0.02182866, requires_grad=False)], Y = -1
X = [tensor(-0.2970185, requires_grad=False), tensor(0.02037128, requires_grad=False)], Y = -1
X = [tensor(-0.37133426, requires_grad=False), tensor(-0.15917612, requires_grad=False)], Y = -1
X = [tensor(-0.40726716, requires_grad=False), tensor(-0.19321337, requires_grad=False)], Y = -1
X = [tensor(-0.19190603, requires_grad=False), tensor(0.26460569, requires_grad=False)], Y =  1


In [28]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [29]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.0788264 | Accuracy: 0.5500000 
Iter:     2 | Cost: 1.3199408 | Accuracy: 0.5500000 
Iter:     3 | Cost: 1.2014018 | Accuracy: 0.5500000 
Iter:     4 | Cost: 0.9852871 | Accuracy: 0.5375000 
Iter:     5 | Cost: 0.9622816 | Accuracy: 0.5125000 
Iter:     6 | Cost: 0.9886902 | Accuracy: 0.5062500 
Iter:     7 | Cost: 1.0063923 | Accuracy: 0.5062500 
Iter:     8 | Cost: 1.0624727 | Accuracy: 0.5000000 
Iter:     9 | Cost: 0.9729599 | Accuracy: 0.5000000 
New best
Iter:    10 | Cost: 0.9467489 | Accuracy: 0.5562500 
New best
Iter:    11 | Cost: 1.0541310 | Accuracy: 0.5687500 
Iter:    12 | Cost: 1.0350467 | Accuracy: 0.5625000 
Iter:    13 | Cost: 0.9910237 | Accuracy: 0.5625000 
New best
Iter:    14 | Cost: 0.9110292 | Accuracy: 0.5875000 
Iter:    15 | Cost: 1.0303047 | Accuracy: 0.5062500 
Iter:    16 | Cost: 1.2872435 | Accuracy: 0.5062500 
Iter:    17 | Cost: 1.2680755 | Accuracy: 0.5062500 
Iter:    18 | Cost: 1.0499880 | Accuracy: 0.5062500 
Iter:    1

In [30]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [31]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 0.8896821085143293, Accuracy: 65.0%


In [32]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,-1.0,1.0
1,-1.0,-1.0
2,1.0,1.0
3,-1.0,-1.0
4,-1.0,1.0
5,-1.0,-1.0
6,-1.0,1.0
7,-1.0,-1.0
8,-1.0,-1.0
9,-1.0,-1.0


In [33]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       1.00      0.61      0.76        36
         1.0       0.22      1.00      0.36         4

    accuracy                           0.65        40
   macro avg       0.61      0.81      0.56        40
weighted avg       0.92      0.65      0.72        40

0.2222222222222222
1.0
0.3636363636363636
0.8055555555555556
